In [ ]:
# load all libraries
import pandas as pd
import ncbi.datasets

In [ ]:
# Start a Datasets gene API instance
api_client = ncbi.datasets.ApiClient()
ds_gene_instance = ncbi.datasets.GeneApi(api_client)

# Retrieve gene summaries for the three genes using NCBI Gene IDs
gene_summary = ds_gene_instance.gene_metadata_by_id([100699762])

# Look up the symbols, chromosome number and SwissProt accession for each gene ,84347,140724,140725,140726,140727,140728,140729,501675,688285
def report_on_gene_descriptors(gene_summary, leader='\t', report_errors=True):
    if report_errors:
        for message in gene_summary.messages or []:
            print(f'{leader}Error for: ({",".join(message.error.invalid_identifiers)})')
            print(f'{leader}{leader}Reason: ({message.error.reason})')

    if not gene_summary.genes:
        print(f'{leader}No genes found')
        return

    for gene in map(lambda g: g.gene, gene_summary.genes):
        print(f'{gene.symbol} (GeneID: {gene.gene_id}), Chromosome: {gene.chromosomes}, SwissProt: {gene.swiss_prot_accessions}')

report_on_gene_descriptors(gene_summary)



In [ ]:
import pandas as pd

def get_acc(species):
    acc = []
    df = pd.read_excel("C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\All_species.xlsx", sheet_name=species)
    for i in df["accession_number"]:
        acc.append(i)
    return acc




In [ ]:
#search location by gene id and protein accession number

acc = ["XP_003965054.2","XP_003978708.1"]
geneID = [101064909,101061612]

def searchlocation(acc, geneID):
    api_client = ncbi.datasets.ApiClient()
    ds_gene_instance = ncbi.datasets.GeneApi(api_client)
    gene_summary = ds_gene_instance.gene_metadata_by_id([geneID])

    for i in gene_summary.genes:
        chromosome = (i.gene.chromosomes[0])
        complement = i.gene.orientation
        symbol = i.gene.symbol
        cdna = ""
        start = ""
        stop = ""
        complement = ""
        accession_version = ""
        cdnastart = ""
        cdnastop = ""
        exon = []
        exonstart = 0
        
        if not i.gene.transcripts:
            pass
        else:
            for l in i.gene.transcripts:
                if not l.protein:
                    pass
                elif l.protein.accession_version == acc:
                    accession_version = i.gene.transcripts[i.gene.transcripts.index(l)].genomic_range.accession_version
                    cdna = i.gene.transcripts[i.gene.transcripts.index(l)].accession_version


                    cdnastart = (i.gene.transcripts[i.gene.transcripts.index(l)].cds.range[0].begin)
                    cdnastop = (i.gene.transcripts[i.gene.transcripts.index(l)].cds.range[0].end)


                    start = (i.gene.transcripts[i.gene.transcripts.index(l)].genomic_range.range[0]['begin'])
                    stop = (i.gene.transcripts[i.gene.transcripts.index(l)].genomic_range.range[0]['end'])
                    complement = (i.gene.transcripts[i.gene.transcripts.index(l)].genomic_range.range[0]['orientation'])

                    exon = [len(i.gene.transcripts[i.gene.transcripts.index(l)].genomic_locations[0].exons)]
                    for num in range (len(i.gene.transcripts[i.gene.transcripts.index(l)].genomic_locations[0].exons)):
                        exon.append(i.gene.transcripts[i.gene.transcripts.index(l)].genomic_locations[0].exons[num])
                    exonstart = int(exon[1]['end'])-int(exon[1]['begin'])-int(cdnastart)
                    tempexon = int(exonstart)
                    print(tempexon)

                else:
                    pass

        if str(complement)=="minus":
            location = symbol + "," + acc + ",\"Chromosome: " + chromosome + "; " + accession_version + "(" + start + ".." + stop + ", complement)\"," + cdna + ", " + cdnastart + ", " + cdnastop + ", " + str(exonstart)
            for num in range (2,len(exon)-1):
                tempexon += int(exon[num]['end'])-int(exon[num]['begin'])
                location += "," + str(tempexon)
        elif str(complement) != "minus":
            location = symbol +  "," + acc + ",Chromosome: " + chromosome + "; " + accession_version + "(" + start + ".." + stop + ")," + cdna + "," + cdnastart + ", " + cdnastop + ", " + str(exonstart)
            for num in range (2,len(exon)-1):
                tempexon += int(exon[num]['end'])-int(exon[num]['begin'])
                location += "," + str(tempexon)
        return location

for i in range(len(acc)):
    print(searchlocation(acc[i], geneID[i]))


In [ ]:
def writecsvtxt(acc,geneID):
    temp = []
    for i in range(len(acc)):
        temp.append(searchlocation(acc[i], geneID[i]))
    return temp


txt = writecsvtxt(acc,geneID)

for i in txt:
    print(i)

In [ ]:
### output to csv
def outputcsv(txt):
    file = open("output.csv",'w')
    file.write("protein,accession,location,cDNAs,begin,end,exon_boundary1,exon_boundary2,exon_boundary3,exon_boundary4,exon_boundary5,exon_boundary6,exon_boundary7,exon_boundary8\n")
    for i in range(len(txt)):
        if i != len(txt)-1:
            file.write(txt[i] + "\n")
        else:
            file.write(txt[i])
    file.close()
outputcsv(txt)

In [ ]:
### get begin and end
def get_start_end():
    df = pd.read_csv("output.csv")
    return list(df['begin']), list(df['end']),list(df['cDNAs'])

start = get_start_end()[0]
stop = get_start_end()[1]
cDNAs = get_start_end()[2]

In [ ]:
####get sequence

import re
import pandas as pd
from urllib.request import urlopen
import os


def AccesiontoID(accession_num):
    url = "https://www.ncbi.nlm.nih.gov/protein/"+str(accession_num)+"?report=gilist&log$=seqview&format=text"
    df = urlopen(url)
    for lines in df:
        lines = lines.decode("utf-8")
        if "<pre>" in lines:
            id = lines[5:].rstrip()
    return id

def get_sequence(geneid,start,stop):
    temp = ""
    url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=fasta&id=" + geneid + "&from=" + str(start) + "&to=" + str(stop) + "&extrafeat=null&conwithfeat=on&hide-cdd=on"
    file = urlopen(url)
    for line in file:
        temp += line.decode("utf-8")
    return temp

for i in range(len(cDNAs)):
    txt = get_sequence(AccesiontoID(cDNAs[i]),start[i],stop[i])
    print(txt)

In [ ]:
path = "C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\New folder\\"

def createfilename(txt):
    path = "C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\New folder\\"
    pattern = "(?<=\\s).+"
    temp = re.findall(pattern,txt)
    temp = re.sub("[;:]","",temp[0])
    return path + temp + ".txt"

def writetxt(path,txt):
    with open (path,"w") as obj:
        obj.write(txt)


temp = ""
for i in range(len(cDNAs)):
    txt = get_sequence(AccesiontoID(cDNAs[i]),start[i],stop[i])
    temp+=txt
    writetxt(createfilename(txt),txt)

writetxt("C:\\Users\\junjie\\Desktop\\lab\\work\\Myfile\\New folder\\" + "all.txt",temp)

In [ ]:
gene_summary

In [ ]:
from pdf2image import convert_from_path
pages = convert_from_path('C:\\Users\\junjie\\Desktop\\推甄\\陽明交通大學(陽明)\\自傳signed.pdf', 1000)

for page in pages:
    page.save('C:\\Users\\junjie\\Desktop\\推甄\\陽明交通大學(陽明)\\自傳signed.jpg', 'JPEG')

In [ ]:
#search location by gene id and protein accession number

acc = ["XP_003456412.1","XP_005463339.1","XP_005463337.1"]
geneID = [100698262,100696764,100696764]

for i in range(len(acc)):
    print(searchlocation(acc[i], geneID[i]))

# Gene structure and transmembrane domains analysis

In [10]:
# read data from excel

import pandas as pd

dataf = pd.read_excel("data\\transmembrane\\Book1.xlsx", sheet_name = "Sheet1")
textname = []
nuclen = []
aalen = []
exonboun = []
trans = []
templist = []

for i in range(0,len(dataf["species"])):
    textname.append(dataf["species"][i]+" "+dataf["protein"][i])
    templen = int(dataf["end"][i])-int(dataf["begin"][i])+1
    nuclen.append(templen)
    aalen.append(templen/3)
    for j in range(1,6):
        templist.append(dataf["exon_boundary"+str(j)][i])
    templist = [x for x in templist if x ==x ]
    exonboun.append(templist)
    templist = []
    for k in range(1,6):
        templist.append(dataf["trans"+str(k)+"_start"][i])
        templist.append(dataf["trans"+str(k)+"_end"][i])
    templist = [x for x in templist if x ==x ]
    trans.append(templist)
    templist = []


In [12]:
# create figure

from PIL import Image, ImageDraw, ImageFont

newImage = Image.new('RGBA', (2000,2500), 'White')
drawObj = ImageDraw.Draw(newImage)

drawstart = 450
recx1 = 5
recx2 = 15
recy1 = 20
recy2 = 40
texty = 15
triy1 = 50
triy2 = triy1+30
gap = 180
adjust = 1.5

fontInfo = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf',45)

for item in range(0,len(dataf)):

    drawObj.text((20,texty), textname[item], fill='Black', font=fontInfo)

    drawObj.rectangle((drawstart,recy1,drawstart+adjust*aalen[item],recy2),fill='lightgrey')

    for i in range (0,len(trans[item]),2):
        drawObj.rectangle((drawstart+round(adjust*trans[item][i]),recx1,drawstart+round(adjust*trans[item][i+1]),recx2),fill='Black')
    
    for i in range (0,len(exonboun[item])):
        drawObj.polygon([(450+(round(exonboun[item][i]/nuclen[item]*adjust*aalen[item]))-10,triy2),(450+(round(exonboun[item][i]/nuclen[item]*adjust*aalen[item])),triy1),(450+(round(exonboun[item][i]/nuclen[item]*adjust*aalen[item]))+10,triy2)],fill='Black')

    texty += gap
    recy1 += gap
    recy2 += gap
    recx1 += gap
    recx2 += gap
    triy1 += gap
    triy2 += gap


drawObj.polygon([(950-10,texty+35),(950,texty+5),(950+10,texty+35)],fill='Black')
drawObj.text((1000,texty), ": Exon boundaries", fill='Black', font=fontInfo, align='right')

drawObj.rectangle((925,texty+90,970,texty+100),fill='Black')
drawObj.text((1000,texty+70), ": Predicted transmembrane segments", fill='Black', font=fontInfo, align='right')

newImage.save("structureout.png")


# Syntenic Block analysis

In [13]:
#read data from csv

import pandas as pd

class Chromosome:
    def __init__(self, species, chr):
        self.species = species
        self.chr = chr

    def read_csv(self):
        dataf = pd.read_csv("data\\syntenic\\"+self.species + " C" + str(self.chr) + ".csv", skiprows=[0])
        return dataf

def get_df(cdict = {"Homo sapiens":[16,17,19,22], "Mus musculus":[7,11,15,16]}):
    specieslist = []
    temp = []
    count = 1
    for i in cdict:
        for j in cdict[i]:
            exec(f"c{count} = Chromosome(i, j)")
            exec(f"df{count} = c{count}.read_csv()")
            exec(f"temp.append(df{count})")
            count += 1
        specieslist.append(temp)
        temp = []
    return specieslist

In [14]:
#get neighbour gene

def get_neighbour(dataf,n):
    address = []
    distance = n
    neighbour_gene = []
    for i in range(len(dataf["Gene symbol"])):
        if "CACNG" in dataf["Gene symbol"][i].upper():
            for j in range (0,distance+1):
                if i-j>=0:address.append(i-j)
                if i+j<len(dataf["Gene symbol"]):address.append(i+j)
                
        elif "TMEM114" in dataf["Gene symbol"][i].upper():
            for j in range (0,distance+1):
                if i-j>=0:address.append(i-j)
                if i+j<len(dataf["Gene symbol"]):address.append(i+j)
        elif "TMEM235" in dataf["Gene symbol"][i].upper():
            for j in range (0,distance+1):
                if i-j>=0:address.append(i-j)
                if i+j<len(dataf["Gene symbol"]):address.append(i+j)

    address=sorted(set(address))

    for i in address:
        neighbour_gene.append(dataf["Gene symbol"][i])
    return neighbour_gene

def get_neighbourlist(specieslist,n):
    temp = []
    neighbour_gene = []
    for i in range(len(specieslist)):
        for j in range(len(specieslist[i])):
            temp.append(get_neighbour(specieslist[i][j],n))
        neighbour_gene.append(temp)
        temp = []   
    return neighbour_gene

In [15]:
cdict = {"Homo sapiens":[16,17,19,22], "Mus musculus":[7,11,15,16], "Canis lupus familiaris":[6,9,10,1]}
common_name = ["Human", "Mouse", "Dog"]
neighbour_gene = get_neighbourlist(get_df(cdict),10)

In [16]:
#count score

from itertools import chain
from collections import Counter

neighbour_gene_list = list(chain.from_iterable(list(chain.from_iterable(neighbour_gene))))
for i in range(len(neighbour_gene_list)):
    neighbour_gene_list[i] = neighbour_gene_list[i].upper()

scoredict = Counter(neighbour_gene_list)

In [17]:
#arrange output data

def filter_score(scoredict,score):
    templist = []
    for i in scoredict:
        if scoredict[i] >= score:
            templist.append(i)
    return templist

def output():
    temp = []
    templist = []
    templist1 = []
    for i in range (len(get_df(cdict))):
        for j in range (len(get_df(cdict)[i])):
            for k in get_df(cdict)[i][j]["Gene symbol"]:
                if k.upper() in filter_score(scoredict,3):
                    temp.append(k)
            templist.append(temp)
            temp = []
        templist1.append(templist)
        templist = []
    return templist1

outputdata = output()



In [29]:
# draw

from PIL import Image, ImageDraw, ImageFont

startpointx = 250
startpointy = 300
textx1 = 200
texty1 = 50
textx2 = 0
texty2 = 300

fontInfo1 = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf',150)
fontInfo2 = ImageFont.truetype('C:\Windows\Fonts\calibri.ttf',50)

newImage = Image.new('RGBA', (10000,10500), 'White')
drawObj = ImageDraw.Draw(newImage)
# drawObj.rectangle((150,startpointx+50,100,startpointx+100),fill='black')
# drawObj.rectangle((150,startpointx+100,100,startpointx+150),outline='black')
# startpointx+=100
for i in range (len(outputdata)):
    drawObj.text((textx1, texty1),common_name[i],fill="black",font=fontInfo1)
    textx1 += 1000
    for j in range (len(outputdata[i])):
        # drawObj.text((startpointx-50,texty), "asdf", fill='Black', font=fontInfo)
        drawObj.rectangle((startpointx,startpointy,startpointx+30,len(outputdata[i][j])*50+startpointy),fill='black')
        startpointy+=300+len(outputdata[i][j])*50
    startpointy = 500
    startpointx += 1000


startpointx = 400
texty2,startpointy = 300,300
for i in range (len(outputdata)):
    for j in range (len(outputdata[i])):
        for k in range (len(outputdata[i][j])):
            # drawObj.text((textx2, texty2),outputdata[i][j][k],fill="black",font=fontInfo2)
            drawObj.text((startpointx-100, texty2),outputdata[i][j][k],fill="black",font=fontInfo2)
            texty2+=50
        startpointy+=300+len(outputdata[i][j])*50
        texty2 = startpointy
    texty2,startpointy = 300,300
    startpointx += 1000
        
newImage.save("out.png")
# newImage.show()

In [45]:
outputdata

[[['ALG1',
   'EEF2KMT',
   'RBFOX1',
   'TMEM114',
   'METTL22',
   'ABAT',
   'TMEM186',
   'PMM2',
   'CARHSP1',
   'USP7',
   'GRIN2A',
   'USP31',
   'SCNN1G',
   'SCNN1B',
   'COG7',
   'GGA2',
   'EARS2',
   'UBFD1',
   'NDUFAB1',
   'PALB2',
   'DCTN5',
   'PLK1',
   'ERN2',
   'CHP2',
   'PRKCB',
   'CACNG3',
   'RBBP6',
   'TNRC6A',
   'SLC5A11',
   'ARHGAP17',
   'LCMT1',
   'AQP8',
   'ZKSCAN2'],
  ['RGS9',
   'AXIN2',
   'CEP112',
   'APOH',
   'PRKCA',
   'CACNG5',
   'CACNG4',
   'CACNG1',
   'HELZ',
   'PSMD12',
   'PITPNC1',
   'NOL11',
   'BPTF',
   'KPNA2',
   'TNRC6C',
   'TMC6',
   'TMC8',
   'SYNGR2',
   'TK1',
   'AFMID',
   'BIRC5',
   'TMEM235',
   'SOCS3',
   'PGS1',
   'DNAH17',
   'CYTH1',
   'USP36',
   'TIMP2',
   'CANT1',
   'C1QTNF1'],
  ['NLRP12',
   'MYADM',
   'PRKCG',
   'CACNG7',
   'CACNG8',
   'CACNG6',
   'OSCAR',
   'NDUFA3',
   'TFPT',
   'PRPF31',
   'CNOT3',
   'LENG1',
   'TMC4',
   'MBOAT7',
   'TSEN34',
   'RPS9'],
  ['MYH9',
   'TXN2',
  